Description: This program detects if an email is spam or not

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [ ]:
#Load the data
from google.colab import files
uploaded = files.upload()

Saving emails.csv to emails.csv


In [ ]:
#Read the CSV file
df = pd.read_csv('emails.csv')

#Print the first 5 rows of data
df.head(5)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [ ]:
#Print the shape(Get the number of rows and colums)
df.shape

(5728, 2)

In [ ]:
#Get the column names 
df.columns

Index(['text', 'spam'], dtype='object')

In [ ]:
#Check for duplicates and remove them
df.drop_duplicates(inplace = True)

In [ ]:
#Show the new shape (number of rows and columns)
df.shape

(5695, 2)

In [ ]:
#Show the number of missing (NAN, NaN, na) data for each column
df.isnull().sum()

text    0
spam    0
dtype: int64

In [ ]:
#Dowload the stopwords package
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def processText(text):
  #1 remove the punctuation
  #2 remove stopwords
  #3 return a list of clean text words

#1
  noPunctuation = [ char for char in text if char not in string.punctuation]
  noPunctuation = ''.join(noPunctuation)

#2
  cleanWords = [word for word in noPunctuation.split() if word.lower() not in stopwords.words('english')]

#3
  return cleanWords

In [ ]:
#Show the tokenization ( a list of tokens also called lemmas)

df['text'].head().apply(processText)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

In [ ]:
#Example

message4 = 'hello world hello hello world play'
message5 = 'test test test test one hello'

print(message4)
print()

#Convert the text to matrix of token counts

from sklearn.feature_extraction.text import CountVectorizer
bow4 = CountVectorizer(analyzer=processText).fit_transform([[message4],[message5]])
print(bow4)
bow4.shape

hello world hello hello world play

  (0, 0)	3
  (0, 4)	2
  (0, 2)	1
  (1, 0)	1
  (1, 3)	4
  (1, 1)	1


(2, 5)

In [ ]:
#Convert a collection of text to a matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer = processText).fit_transform(df['text'])

In [ ]:
#Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_bow, df['spam'],test_size=0.20, random_state=0)

In [ ]:
#Get the shape of message_bow
message_bow.shape

(5695, 37229)

In [ ]:
#Create and train the Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train, y_train)

In [ ]:
#Print the predictions
print(classifier.predict(X_train))

#Prin the actual values
print(y_train.values)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [ ]:
#Evaluate the model on the training dataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

    accuracy                           1.00      4556
   macro avg       0.99      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556



In [ ]:
print('Confusion Matrix::\n',confusion_matrix(y_train, pred))

print('Accuracy Score::\n', accuracy_score(y_train, pred))

Confusion Matrix::
 [[3445   12]
 [   1 1098]]
Accuracy Score::
 0.9971466198419666


In [ ]:
#Evaluate the model on the test dataset
print(classifier.predict(X_test))
print(y_test.values)

pred = classifier.predict(X_test)
print('\nClassification Report::\n',classification_report(y_test, pred))
print('\nConfusion Matrix::\n', confusion_matrix(y_test,pred))
print('\nAccuracy Score::\n', accuracy_score(y_test, pred))

[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]

Classification Report::
               precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139


Confusion Matrix::
 [[862   8]
 [  1 268]]

Accuracy Score::
 0.9920983318700615
